In [2]:

import os
import dask
import panel
import pandas as pd
import earthaccess
import matplotlib.pyplot as plt
import nisardev as nisar
import grimpfunc as grimp
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
ProgressBar().register()
dask.config.set(num_workers=2)  # Avoid problems with too many open connections at NSIDC
panel.extension() 

/opt/anaconda3/envs/greenlandMapping/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
earthaccess.login()

In [4]:
env = dict(GDAL_HTTP_COOKIEFILE = os.path.expanduser('~/.grimp_download_cookiejar.txt'),
            GDAL_HTTP_COOKIEJAR = os.path.expanduser('~/.grimp_download_cookiejar.txt'))
os.environ.update(env)

In [5]:
time = ("2015-01-01", "2021-12-31")
bounding = (-51.0, 68.9, -48.5, 69.5)

results = earthaccess.search_data(
    short_name="NSIDC-0766",
    version="2",
    temporal=time,
    bounding_box=bounding,
)

In [9]:
vv = []

for gran in results:
    for url in gran.data_links():
        if "_vv_" in url and url.endswith(".tif"):
            vv.append(url)


baseNames = [x.split("/")[-1] for x in vv]

print(baseNames)
print("Number of vv URLs:", len(vv))


['GL_vel_mosaic_s1cycle_01Jan15_12Jan15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_13Jan15_24Jan15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_25Jan15_05Feb15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_06Feb15_17Feb15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_18Feb15_01Mar15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_02Mar15_13Mar15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_14Mar15_25Mar15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_26Mar15_06Apr15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_07Apr15_18Apr15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_19Apr15_30Apr15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_01May15_12May15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_13May15_24May15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_25May15_05Jun15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_06Jun15_17Jun15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_18Jun15_29Jun15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_30Jun15_11Jul15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_12Jul15_23Jul15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_24Jul15_04Aug15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_05Au

In [10]:
download_dir = "GrIMP_JI_test" 
os.makedirs(download_dir, exist_ok=True)

local_files = earthaccess.download(vv, download_dir)


QUEUEING TASKS | :   0%|          | 0/373 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/373 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/373 [00:00<?, ?it/s]

In [35]:
myCogs = []
for f in local_files:
    name = Path(f).name
    if name.endswith(".tif") and "_vv_" in name and "browse" not in name:
        myCogs.append(str(f))

baseNames = [x.split('/')[-1] for x in myCogs]
print(baseNames)

['GL_vel_mosaic_s1cycle_01Jan15_12Jan15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_13Jan15_24Jan15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_25Jan15_05Feb15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_06Feb15_17Feb15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_18Feb15_01Mar15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_02Mar15_13Mar15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_14Mar15_25Mar15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_26Mar15_06Apr15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_07Apr15_18Apr15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_19Apr15_30Apr15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_01May15_12May15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_13May15_24May15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_25May15_05Jun15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_06Jun15_17Jun15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_18Jun15_29Jun15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_30Jun15_11Jul15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_12Jul15_23Jul15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_24Jul15_04Aug15_vv_v02.0.tif', 'GL_vel_mosaic_s1cycle_05Au

In [36]:
# Define bounding box for subsetting the data
bbox = {'minx': -250000, 'miny': -2250000, 'maxx': -150000, 'maxy': -2150000}

# Create a velocity series object with the COGs searched for above
jiVelSeries = nisar.nisarVelSeries()
jiVelSeries.readSeriesFromTiff(myCogs, url=False, readSpeed=False)

# Subset the velocity series to the bounding box
jiVelSeries.subSetVel(bbox)

RasterioIOError: GrIMP_JI_test/GL_vel_mosaic_s1cycle_01Jan15_12Jan15_vv_v02.0.tif.tif: No such file or directory